# MLDA experiment in Basin

### Classes and modules

In [2]:
#Lets have matplotlib "inline"
%matplotlib inline

import os
import sys

#Import packages we need
import numpy as np
import datetime
from IPython.display import display
import copy

#For plotting
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams["lines.color"] = "w"
plt.rcParams["text.color"] = "w"
plt.rcParams["axes.labelcolor"] = "w"
plt.rcParams["xtick.color"] = "w"
plt.rcParams["ytick.color"] = "w"

plt.rcParams["image.origin"] = "lower"

import pycuda.driver as cuda

In [3]:
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%dT%H_%M_%S")


GPU Ocean-modules:

In [4]:
from gpuocean.utils import IPythonMagic, Common
from gpuocean.SWEsimulators import CDKLM16, ModelErrorKL

In [5]:
%cuda_context_handler gpu_ctx

In [6]:
gpu_stream = cuda.Stream()

Utils

In [7]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))
from utils.BasinInit import *
from utils.BasinPlot import *

In [8]:
truth_path = "/home/florianb/havvarsel/multilevelDA/scripts/DataAssimilation/BasinTruth/2023-06-22T13_47_48"

## Case 

In [9]:
ls = [6, 7, 8, 9]

In [10]:
from utils.BasinParameters import * 

In [11]:
args_list = []

for l in ls:
    lvl_grid_args = initGridSpecs(l)
    args_list.append( {
        "nx": lvl_grid_args["nx"],
        "ny": lvl_grid_args["ny"],
        "dx": lvl_grid_args["dx"],
        "dy": lvl_grid_args["dy"],
        "gpu_ctx": gpu_ctx,
        "gpu_stream": gpu_stream,
        "boundary_conditions": Common.BoundaryConditions(2,2,2,2)
        } )

In [12]:
data_args_list = []
for l_idx in range(len(args_list)): 
    data_args_list.append( make_init_steady_state(args_list[l_idx], a=steady_state_bump_a, bump_fractal_dist=steady_state_bump_fractal_dist) )


In [13]:
Nes = [100, 50, 20, 5]

In [14]:
from gpuocean.ensembles import MultiLevelOceanEnsemble
MLOceanEnsemble = MultiLevelOceanEnsemble.MultiLevelOceanEnsembleCase(Nes, args_list, data_args_list, sample_args, make_sim,
                            init_model_error_basis_args=init_model_error_basis_args, 
                            sim_model_error_basis_args=sim_model_error_basis_args, sim_model_error_timestep=sim_model_error_timestep)

from gpuocean.dataassimilation import MLEnKFOcean
MLEnKF = MLEnKFOcean.MLEnKFOcean(MLOceanEnsemble)

In [15]:
MLOceanEnsemble.stepToObservation(900.0)

In [16]:
from utils.BasinSL import *

def g_functionalExp(SL_state):
    """
    L_g functional as in notation of Kjetil's PhD thesis.
    This should be the functional that is under investigation for the variance level plot

    Input a ndarray of size (3, ny, nx, Ne)

    Returns a ndarray of same size as SL_state (3, ny, nx, Ne)
    """
    return SL_state

def g_functionalVar(SL_state):
    """
    L_g functional as in notation of Kjetil's PhD thesis.
    This should be the functional that is under investigation for the variance level plot

    Input a ndarray of size (3, ny, nx, Ne)

    Returns a ndarray of same size as SL_state (3, ny, nx, Ne)
    """
    return (SL_state - np.mean(SL_state, axis=-1)[:,:,:,np.newaxis])**2
    
def L2norm(field, lvl_grid_args):
    """
    integral_D(f dx)
    where D are uniform finite volumes

    Input:
    field           - ndarray of shape (3,ny,nx,..)
    lvl_grid_args   - dict with nx, ny and dx, dy information

    Output:
    L2norm          - ndarray of shape (3,...)
    """
    # assert field.shape[1:3] == (lvl_grid_args["ny"], lvl_grid_args["nx"]), "field has wrong resolution"
    return np.sqrt(np.sum((field)**2 * lvl_grid_args["dx"]*lvl_grid_args["dy"], axis=(1,2)))



In [17]:
ML_state = MLOceanEnsemble.download()

In [50]:
centers = []
for l_idx in range(len(ls)): 
    center_N = int(args_list[l_idx]["nx"]/8)
    center_x = int(args_list[l_idx]["nx"]/2)
    center_y = int(args_list[l_idx]["ny"]/2)
    centers.append( np.s_[:, center_y-center_N:center_y+center_N, center_x-center_N:center_x+center_N,:] )

In [41]:
prior_gExp = np.zeros((len(ls),3))
prior_gVar = np.zeros((len(ls),3))
prior_gExp_diff = np.zeros((len(ls),3))
prior_gVar_diff = np.zeros((len(ls),3))

In [51]:
for l_idx in range(1,len(ls)):
    prior_gExp[l_idx] = L2norm(np.var(g_functionalExp(ML_state[l_idx][0][centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])
    prior_gVar[l_idx] = L2norm(np.var(g_functionalVar(ML_state[l_idx][0][centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])

    prior_gExp_diff[l_idx] = L2norm(np.var(g_functionalExp(ML_state[l_idx][0][centers[l_idx]]) - g_functionalExp(ML_state[l_idx][1].repeat(2,1).repeat(2,2)[centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])
    prior_gVar_diff[l_idx] = L2norm(np.var(g_functionalVar(ML_state[l_idx][0][centers[l_idx]]) - g_functionalVar(ML_state[l_idx][1].repeat(2,1).repeat(2,2)[centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])


In [52]:
prior_gVar, prior_gVar_diff

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.94273245e+00, 1.76760400e+07, 3.31826400e+07],
        [2.73457074e+00, 5.47649000e+07, 6.86030400e+07],
        [4.31430912e+00, 5.95542400e+07, 1.57749580e+07]]),
 array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.88380387e-02, 1.92293719e+05, 7.28283562e+05],
        [4.67220321e-03, 1.01467469e+05, 1.75830172e+05],
        [1.17858569e-03, 2.19642422e+04, 1.78624922e+04]]))

Data Assimilation

In [53]:
truth_path = "/home/florianb/havvarsel/multilevelDA/scripts/DataAssimilation/BasinTruth/2023-06-22T13_47_48"

In [54]:
precomp_GC = []
for obs_x, obs_y in zip(obs_xs, obs_ys):
    precomp_GC.append( MLEnKF.GCweights(obs_x, obs_y, r) )

In [55]:
# DA step
true_eta, true_hu, true_hv = np.load(truth_path+"/truth_"+str(int(MLOceanEnsemble.t))+".npy")

for h, [obs_x, obs_y] in enumerate(zip(obs_xs, obs_ys)):
    Hx, Hy = MLOceanEnsemble.obsLoc2obsIdx(obs_x, obs_y)
    obs = [true_eta[Hy,Hx], true_hu[Hy,Hx], true_hv[Hy,Hx]] + np.random.normal(0,R)
    
    prior = copy.deepcopy(MLOceanEnsemble.download())

    ML_K = MLEnKF.assimilate(MLOceanEnsemble, obs, obs_x, obs_y, R, 
                            r=r, obs_var=slice(1,3), relax_factor=relax_factor, 
                            min_localisation_level=0,
                            precomp_GC=precomp_GC[h])

In [60]:
ML_state = MLOceanEnsemble.download()

In [61]:
posterior_gExp = np.zeros((len(ls),3))
posterior_gVar = np.zeros((len(ls),3))
posterior_gExp_diff = np.zeros((len(ls),3))
posterior_gVar_diff = np.zeros((len(ls),3))

In [62]:
for l_idx in range(1,len(ls)):
    posterior_gExp[l_idx] = L2norm(np.var(g_functionalExp(ML_state[l_idx][0][centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])
    posterior_gVar[l_idx] = L2norm(np.var(g_functionalVar(ML_state[l_idx][0][centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])
    
    posterior_gExp_diff[l_idx] = L2norm(np.var(g_functionalExp(ML_state[l_idx][0][centers[l_idx]]) - g_functionalExp(ML_state[l_idx][1].repeat(2,1).repeat(2,2)[centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])
    posterior_gVar_diff[l_idx] = L2norm(np.var(g_functionalVar(ML_state[l_idx][0][centers[l_idx]]) - g_functionalVar(ML_state[l_idx][1].repeat(2,1).repeat(2,2)[centers[l_idx]]),ddof=1, axis=-1), args_list[l_idx])


In [63]:
posterior_gVar, posterior_gVar_diff

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.04965389e+00, 7.73194650e+06, 2.25394180e+07],
        [1.86873877e+00, 3.18739100e+07, 4.41292280e+07],
        [1.76371062e+00, 3.15729640e+07, 1.07291390e+07]]),
 array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.15062678e-02, 9.50481094e+04, 5.16164938e+05],
        [2.55720713e-03, 5.27026484e+04, 1.12557609e+05],
        [2.19191556e-04, 5.49662842e+03, 5.47383398e+03]]))

Comparison

In [64]:
posterior_gVar/prior_gVar

/home/florianb/miniconda3/envs/gpuocean_opendrift/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[       nan,        nan,        nan],
       [0.54029771, 0.43742527, 0.67925331],
       [0.68337554, 0.58201348, 0.6432547 ],
       [0.40880488, 0.53015476, 0.68013741]])

In [65]:
posterior_gVar_diff/prior_gVar_diff

/home/florianb/miniconda3/envs/gpuocean_opendrift/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


array([[       nan,        nan,        nan],
       [0.61079967, 0.49428608, 0.70874171],
       [0.54732361, 0.51940439, 0.64014957],
       [0.18597846, 0.2502535 , 0.3064429 ]])